In [4]:
# -- Setup environment and imports --

# Add 'src' directory to path to import project modules
import sys
from pathlib import Path
sys.path.append(str(Path().resolve().parent / "src"))

from datetime import datetime, timedelta
import numpy as np
import pandas as pd
import eikon as ek
import os

from config import CORN_QUOTES, SOYBEAN_QUOTES, SOYBEAN_PREMIUM_QUOTES

In [5]:
ek.set_app_key('24fc8f862b0540b990c9460a04f724972d5e57f0')

def fetch_full_history(symbol, start_date, end_date, interval="daily"):
    all_data = []
    chunk_start = pd.to_datetime(start_date)
    chunk_end = chunk_start + timedelta(days=3000)

    while chunk_start < pd.to_datetime(end_date):
        chunk_data = ek.get_timeseries(
            symbol,
            fields=["Close"],
            start_date=chunk_start.strftime("%Y-%m-%d"),
            end_date=min(chunk_end, pd.to_datetime(end_date)).strftime("%Y-%m-%d"),
            interval=interval
        )

        if chunk_data is None or chunk_data.empty:
            break

        all_data.append(chunk_data)
        chunk_start = chunk_data.index.max() + timedelta(days=1)
        chunk_end = chunk_start + timedelta(days=3000)

    full_df = pd.concat(all_data)
    full_df = full_df[~full_df.index.duplicated()]
    return full_df


def fetch_and_cache_timeseries(symbol, cache_path, start_date="2000-01-01"):
    hoje = pd.Timestamp.today().normalize()

    if os.path.exists(cache_path):
        df_cache = pd.read_excel(cache_path, index_col=0, parse_dates=True)
        ultima_data = df_cache.index.max().normalize()
        if ultima_data < hoje:
            df_novo = fetch_full_history(
                symbol,
                start_date=(ultima_data + timedelta(days=1)).strftime("%Y-%m-%d"),
                end_date=hoje.strftime("%Y-%m-%d")
            )
            df = pd.concat([df_cache, df_novo])
            df = df[~df.index.duplicated()]
            df.to_excel(cache_path)
        else:
            df = df_cache
    else:
        df = fetch_full_history(
            symbol,
            start_date=start_date,
            end_date=hoje.strftime("%Y-%m-%d")
        )
        df.to_excel(cache_path)

    return df


df_corn = fetch_and_cache_timeseries("Cc1", CORN_QUOTES)
df_soybean = fetch_and_cache_timeseries("Sc1", SOYBEAN_QUOTES)
df_soybean_premium = fetch_and_cache_timeseries("SB-PREMP-C1", SOYBEAN_PREMIUM_QUOTES)
